<a href="https://colab.research.google.com/github/vishnubob/pixace/blob/main/examples/pixace_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pixace

In [ ]:
%load_ext tensorboard

# download and install the latest version of pixace
!sudo pip install -qqq -U \
  https://github.com/vishnubob/pixace/archive/main.zip
  
import pixace

In [ ]:
# download the animalfaces model

pixace.download_model(model_name="animalfaces")

In [ ]:
# generate a new animal face from scratch

predictor = pixace.get_predictor(model_name="animalfaces")
predictor.predict()

In [ ]:
# use two images from the web as image prompts

url_1 = "https://github.com/vishnubob/pixace/raw/main/examples/prompts/prompt_1.jpg"
url_2 = "https://github.com/vishnubob/pixace/raw/main/examples/prompts/prompt_2.jpg"
prompts = [url_1, url_2]
predictor.predict(prompts=prompts, cut=512)

In [ ]:
# download emoji image set to use for training

import os
import io
from zipfile import ZipFile
import requests

def download_and_extract(url, out="data"):
  resp = requests.get(url)
  fh = io.BytesIO(resp.content)
  fh.seek(0)
  with ZipFile(fh) as arc:
    arc.extractall(out)

url = "https://github.com/googlefonts/noto-emoji/archive/master.zip"
weight_dir = "model-weights"
out = "emoji-data"
images = f"{out}/noto-emoji-master/png/128"

download_and_extract(url, out)
os.makedirs(weight_dir, exist_ok=True)

%tensorboard --logdir model-weights

In [ ]:
# train a new model on the emoji image dataset
# visualize samples from training by clicking on the tensorboard images tab above 
# this training run will take about 20 minutes
# you should have enough time to make yourself and eat a sandwich

trainer = pixace.get_trainer(model_name="emoji")
trainer.train(
    steps_per_epoch=100,
    images=images,
    batch_size=16,
    n_epochs=5
  )

In [ ]:
# use your new emoji model to generate new emojis

predictor = pixace.get_predictor(model_name="emoji")
predictor.predict(batch_size=2, temperature=(0.9, 1, 1.1))